### Импорт библиотек

In [ ]:
# Импорт библиотек для проведения расчетов
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
# Импорт библиотек для кластеризации
# Документация: https://scikit-learn.org/stable/modules/clustering.html
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram

In [ ]:
# Импорт библиотек для визуализации
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Библиотека для построения Self-Organizing Maps: sklearn-som
# Документация библиотеки: https://sklearn-som.readthedocs.io/en/latest/index.html

# ! python3.7 -m pip install sklearn-som
from sklearn_som.som import SOM

ModuleNotFoundError: ignored

## Загрузка данных

In [ ]:
# Указываем путь к файлу
# При работе с google colab существует 2 варианта загрузки файла:
# 1. Через google drive
# 2. Через левое меню: "файлы" -> "загрузить в сессионное хранилище" -> "скопировать файл" (при наведении на файл)
file_path = '/content/baseball.sas7bdat'

In [ ]:
# Загружаем данные из SAS файла
df = pd.read_sas(file_path, encoding='latin-1')
# Пример данных
df.head()

## Формирование признакового пространства

In [ ]:
# Сформируем признаковое пространство:
# 1. YrMajor - число лет в лиге
# 2. Признаки с префиксом "n" - результаты 1986 года 
# 3. Признаки с префиксом "сr"  - результаты за всю карьеру

results_1986 = [c for c in df.columns if c[0] == "n"]
results_all = [c for c in df.columns if c[:2] == "Cr"]
sign_features = ['YrMajor'] + results_1986 + results_all

In [ ]:
# Стандартизация признаков (вставьте свой код)
scaler = StandardScaler()

X = df.loc[:, sign_features]

## Часть 1: Иерархическая кластеризация и K-means

In [ ]:
# Обучим модель Иерархической кластеризации
clustering = AgglomerativeClustering(compute_distances = True, distance_threshold = 0.0, n_clusters = None).fit(X)

In [ ]:
# Визуализация дендрограммы
def plot_dendrogram(model, **kwargs):
    # Создаем матрицу связности

    # Рассчитываем количество наблюдений в каждой вершине
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # Лист
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    fig, axes = plt.subplots(1, 1, figsize=(12, 10))
    dendrogram(linkage_matrix, **kwargs, show_leaf_counts = True)

plot_dendrogram(clustering, truncate_mode="level", p=3)

In [ ]:
# Выберем число кластеров на основе псевдо критерия Фишера

def sum_dist_to_center(X):
    center = np.mean(X, axis = 0)
    return ((X - center)**2).values.sum()

def choose_num_clusters(X, max_clust = 30):
    N = X.shape[0]
    Q = sum_dist_to_center(X)
    pseudo_f = np.array([])
    for G in range(2, max_clust):
        clustering = AgglomerativeClustering(compute_distances = True, n_clusters = G).fit(X)
        W = 0
        for l in range(G):
            elems = X[clustering.labels_ == l]
            W += sum_dist_to_center(elems)
        fisher_stat = ((Q - W)/(G - 1))/(W/(N - G))
        pseudo_f = np.append(pseudo_f, fisher_stat)
        
    plt.plot(range(2, max_clust), pseudo_f)
    return np.argmax(pseudo_f)+2

k = choose_num_clusters(X)
clustering = KMeans(n_clusters=k, random_state=0).fit(X)
label = clustering.labels_

### Ответить на следующие вопросы:

#### 1.   Сколько кластеров было выбрано на основе псевдо критерия Фишера?
#### 2.   Используя информацию из графика, ответьте на вопросы:
> 1.   Каково значение критерия для выбранного числа кластеров?
> 2.   А для числа кластеров на один больше и на один меньше?



## Часть 2: SOM

In [ ]:
# Обучаем Self-Organizing Map с размером сетки 3 на 1
baseball_som = SOM(m=3, n=1, dim = X.shape[1])
baseball_som.fit(X.to_numpy())

labels = baseball_som.predict(X.to_numpy())

In [ ]:
## Дополнительно #
## Значения признаков по кластерам удобно визуализировать с помощью barplot
## Нужное раскомментировать

## 1. В терминах matplotlib 
# plt.bar(labels, X["CrBB"])
# plt.ylabel("CrBB")
# plt.xlabel("cluster")

## 2. В терминах seaborn (+ доверительные интервалы)
# sns.barplot(x = labels, y = X["CrBB"], capsize = 0.1)

## Часть 3: Анализ кластеров

## Построить модели:
> Для части 1: выбор количества кластеров на основе иерархической кластеризации с последующим обучение k-means

> Для части 2: обучение SOM с размером сетки 2 на 2 

## Ответить на следующие вопросы для каждой построенной модели
> Вариант I: Какой кластер содержит самых опытных игроков (переменная YrMajor)? 
> Вариант II: Какой кластер содержит самых успешных по хитам в 1986 году игроков (переменная nHits)?
> 1.	Сколько человек попало в этот кластер? 
> 2.    Как зовут самого типичного игрока в кластере вашего варианта (у него должно быть минимальное расстояние до центра вашего кластера)?